# Inventory Server

A common feature of online games is the 'inventory' which provides a list of all of the items 'owned' by an object.

It is frequently used in MMOs.

This project attempts to make scaling this service easier by making the developer work a little harder up front, by using persistent objects. The expected out come is that the datastore is significantly smaller, due to massive inventory and item reuse. Another benefit is caching, the majority of calls to the service can be cached indefinitely.

The service provides the notions of

* A user: A root object that has a collection of inventories
* An inventory: Items that can be collected under a single category, eg: Player attributes, achievements, rock collection
* A metric: A datum in an inventory, eg: First name, Sword of Justice



In [ ]:
import graphviz

d = graphviz.Digraph()
d.node("Inventory")
d.node("User")
d.node("Category")
d.node("Metric")
d.node("Value")
d.edge("User", "Inventory", "*:1")
d.edge("Category", "Inventory", "*:1")
d.edge("Category", "Category", "parent")
d.edge("Inventory", "Metric", "*:*")
d.edge("Value", "Metric", "*:1")
d

But isn't that a bit backwards in places?

Well, one of the things we found building out games was that there is a lot of repetition. Players have very similar patterns and pick up similar items. For a moderately sized poker simulation we rapidly started to approach gigabytes of data that compressed to almost nothing, which gave us the idea to reuse identical objects. 

InventoryServer uses SHA1 to for all identification, so if a category already exists, it is not recreated. We'll explore that a little closer with examples further into this document.

First lets build the server.

## Build

The API was designed to be persistence agnostic. At the momement we have implementions for in memory (essentially a hashmap) and JPA.

Why maintain two storage engines?

Similarly to having tests in both Python and Java they exposed different bugs and have different virtues. For demonstrating the API and running unit tests, the memory server is ideal.

Let us build it.

In [ ]:
%%bash

mvn clean package

In [ ]:
%%html
<iframe src="core/target/site/jacoco/index.html" width="100%" height="300px"/>

In [ ]:
%%html
<iframe src="memory/target/site/jacoco/index.html" width="100%" height="300px"/>

In [ ]:
%%html
<iframe src="jpa/target/site/jacoco/index.html" width="100%" height="300px"/>

## Deploy to AWS

You need to put your AWS credentials in the variables below for this to work.

These credentials must have:

* AmazonEC2FullAccess
* AmazonVPCFullAccess

To create Security Groups, an ELB, a VPC, an internet gateway and an EC2 instance. 

Full details can be seen in the [terraform](deploy-memory/memory.tf)

*DO NOT CHECK IN YOUR TOKENS*

In [ ]:
%env AWS_ACCESS_KEY=<ACCESS KEY HERE>
%env AWS_SECRET_KEY=<SECRET KEY HERE>

In [ ]:
%%bash
pushd deploy-memory
terraform apply -auto-approve -var "aws_access_key=$AWS_ACCESS_KEY" -var "aws_secret_key=$AWS_SECRET_KEY"

In [ ]:
%%bash
pushd deploy-memory
terraform output ip > ip.txt

That should have started an EC2 instance on AWS with the server you created. Now we're going to generate a client and some documentation using swagger.

## Service Tests

In [ ]:
%%bash
rm -rf inventory_server/
rm -rf temp
mkdir temp
cd temp
swagger-codegen generate -l python -i ../meta/src/main/docs/swagger.yml
swagger-codegen generate -l html -i ../meta/src/main/docs/swagger.yml
mv swagger_client ../inventory_server
cd ..


So what did that do?

It took [`swagger.yml`](meta/src/main/docs/swagger.yml) and compiled it into a native python client that we will use for our tests. We use the same yaml to generate the Java client that is the output of the `meta` module, which is used by the Java service tests. 

We use that swagger file to define the API and generate this documentation:

In [ ]:
%%html
<iframe src="temp/index.html" width="100%" height="300px"/>

Why do we like swagger?

It allows us to provably document our API. If you change the swagger in the `meta` module the tests in this file may fail, as may the Java unit tests, which also use swagger to generate clients.


Excellent. Now we're going to use the API that we defined with Swagger to interact with the service. 

In [ ]:
from inventory_server.configuration import Configuration
from inventory_server.apis import DefaultApi
import os

conf = Configuration()
conf.host = "http://%s:5555" % open("deploy-memory/ip.txt", "r").read().strip()
api = DefaultApi()


Let us see what we have in the stored.

In [ ]:
inventories = api.all_inventories()

if 0 != len(inventories):
    raise Exception("Try clearing the DB by restarting the service")


Nothing! Thats not that surprising as the database should always start empty (its ephemeral).

Lets create and store our first inventory.

In [ ]:
from inventory_server.models.inventory import Inventory

inventory = Inventory(category = "test.category")

api.create_inventory(inventory)



That was easy! 

But... did it work?

In [ ]:
moreInventories = api.all_inventories()

if "test.category" != moreInventories[0].category:
    raise Exception("Found ")

Hopefully we found the inventory. Excellent!

OK. On to users. 

In [ ]:
from inventory_server.models.user import User

user = User(name = "Tilda")

saved = api.create_user(user)

if "Tilda" != saved.name:
    raise Exception("The user has the wrong name")

users = api.all_users()

if "Tilda" != users[0].name:
    raise Exception("The user is missing from all users")

So we proved we can create a user.

In [ ]:
from inventory_server.models.metric import Metric

metric = Metric(type="arrows", value="34")

inventory = Inventory(category="test.flarp", metrics=[metric])

user = User(name="Archer")

archer = api.update_inventory_for_user("Archer", "test.flarp", inventory)
if "test.flarp" != archer.category:
    raise Exception("We didn't update the user")

savedArcher = api.find_latest_user("Archer")
if "test.flarp" != savedArcher.inventories[0].category:
    raise Exception("The lastest version of Archer is not updated correctly")

Now we're getting a little more realistic. A user, with an inventory and a metric.

In [ ]:
metric = Metric(type="arrows", value="34")

inventory = Inventory(category="test.flarp", metrics=[metric])

metric2 = Metric(type="Bows", value="1")

inventory2 = Inventory(category="test.floop", metrics=[metric2])

user = User(name="Archer")

api.create_user(user)

inv1 = api.update_inventory_for_user("Archer", "test.flarp", inventory)
inv2 = api.update_inventory_for_user("Archer", "test.floop", inventory2)

inventories = api.all_inventories_for_user("Archer")
if 1 != len([x for x in inventories if x.category == "test.flarp"]):
    raise Exception("You're missing the test.flarp category")
    
if 1 != len([x for x in inventories if x.category == "test.floop"]):
    raise Exception("You're missing the test.floop category")

And finally, one user, two metrics and a couple of inventories. 

## TODO

1. Demonstrate the same tests for the JPA / RDS version of the server
2. Provide performance benchmarks and graphs